# Gallery Example: M/Erl/1 Reentrant Queue

This example demonstrates an M/Erlang/1 queueing system with reentrant routing:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Erlang distribution (Class1), Exponential (Class2)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_merl1_reentrant(): Network {
    """Create M/Erl/1-Reentrant queueing model"""
    val model = Network("M/Erl/1-Reentrant")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    val oclass2 = OpenClass(model, "Class2")
// Exponential arrival for Class1, Class2 is disabled at source
    source.setArrival(oclass1, Exp(1))
    source.setArrival(oclass2, Disabled())
// Erlang service for Class1, Exponential for Class2
    queue.setService(oclass1, Erlang.fit_mean_and_order(0.5, 5))
    queue.setService(oclass2, Exp(3))
// Block 3: topology with class switching (reentrant)
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addClassSwitch(oclass1, oclass2, queue, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_merl1_reentrant()

## About M/Erl/1 Queues

The M/Erl/1 queue combines:
- **Poisson arrivals** (SCV = 1): Standard random arrivals
- **Erlang service** (SCV = 1/k < 1): More regular service times

The Erlang service with order 5 gives SCV = 0.2, meaning service times are quite regular. This typically results in:
- Lower queue lengths than M/M/1 at the same utilization
- More predictable waiting times

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)